# Capstone Project - The Battle of Neighborhoods

# 1. Introduction

I would like to introduce my problem related to opening the tourist office around Hanoi, Vietnam. 

In Hanoi, there are a significant number of individual foreign tourism who do not book a completed tour before their travel. The tourist market is lacking the cheap quality mini trip service such as around Hanoi on bike, food tour or walking tour. I and my college plan to run the business in this sector and we are looking for the place to open 4-5 offices around Hanoi to attract the foreign customer.

My problem is where are the good places to open these tourist service office? 

The requirement of the office area should be:
 - There are many foreign individual tourists around (they could live, have food, drink near there)
 - The distance between the office should be optimal to gather as much as the customer all the office can.

# 2. Data

# 3. Methodology

# 4. Result

# 5. Discussion

# 6. Conclusion